In [45]:
import pandas as pd
# process southern stumpage prices
pricesSouth = pd.read_csv('../data/Timber Prices/prices_south.csv')

# columns 4 through 69 are stumpage prices grouped by product type,
# state abbreviation and state region. for example, sawfl1 is sawtimber prices for
# state of Florida and Florida region 1. we can use the pandas melt function to
# convert these columns into rows

pricesSouth = pricesSouth.melt(
    id_vars=pricesSouth.columns[0:3],
    value_vars=pricesSouth.columns[3:69],
    var_name='product',
    value_name='price'
    )

# split the product column into three columns: product, stateAbbr, priceRegion
pricesSouth['stateAbbr'] = pricesSouth['product'].str[3:5].str.upper()
pricesSouth['priceRegion'] = pricesSouth['product'].str[5:].str.zfill(2)
pricesSouth['product'] = pricesSouth['product'].str[:3]

# drop product if equal to 'pre'
pricesSouth = pricesSouth[pricesSouth['product'] != 'pre']

# change 'saw' to 'Sawtimber' and 'pul' to 'Pulpwood'
pricesSouth['product'] = pricesSouth['product'].replace({'saw': 'Sawtimber',
                                                         'plp': 'Pulpwood'})

# change 'pine' and 'oak' to 'Pine' and 'Oak'
pricesSouth['type'] = pricesSouth['type'].replace({'pine': 'Pine',
                                                    'oak': 'Oak'})

# change column 'type' to 'priceSpecies'
pricesSouth.rename(columns={'type': 'priceSpecies'}, inplace=True)

# add state fips code
state_fips = {'AL': '01', 'AR': '05', 'FL': '12', 'GA': '13', 'KY': '21',
              'LA': '22', 'MS': '28', 'NC': '37', 'OK': '40', 'SC': '45',
              'TN': '47', 'TX': '48', 'VA': '51'}
pricesSouth['statecd'] = pricesSouth['stateAbbr'].map(state_fips)

# aggregate prices by year, state, priceRegion, product, priceSpecies
pricesSouth = pricesSouth.groupby(['statecd', 'stateAbbr', 'priceRegion',
                                  'priceSpecies', 'product'])['price'].mean().reset_index()

# convert price to dollars per cubic foot from dollars per ton
# 1 ton = 40 cubic feet
pricesSouth['price'] = pricesSouth['price'] / 40
pricesSouth.rename(columns={'price': 'cuftPrice',
                            'product': 'Product'}, inplace=True)

# group by priceSpecies and product to return the mean over all years
pricesSouth = pricesSouth.groupby(['statecd', 'stateAbbr', 'priceRegion',
                                  'priceSpecies', 'Product'])['cuftPrice'].mean().reset_index()

# print the first 5 rows of the pricesSouth dataframe
# print(pricesSouth.head())

## Southern Cut List

In [46]:
# load harvest species list
harvestSpecies = pd.read_csv('../data/Southern harvested tree species.csv')

# drop if ESTIMATE is 0
harvestSpecies = harvestSpecies[harvestSpecies['ESTIMATE'] != 0]

# drop if Estimate is NaN
harvestSpecies = harvestSpecies.dropna(subset=['ESTIMATE'])

# characters 7 and 8 are of GRP2 is state fip code. extract and name columm statecd
harvestSpecies['statecd'] = harvestSpecies['GRP2'].str[6:8]

# 12-14 of GRP1 is the species code. extract and name column spcd
harvestSpecies['spcd'] = harvestSpecies['GRP1'].str[11:14]

# convert spcd to int64
harvestSpecies['spcd'] = harvestSpecies['spcd'].astype('int64')

# rename ESTIMATE to volume
harvestSpecies.rename(columns={'ESTIMATE': 'volume'}, inplace=True)

# keep only the columns we need
harvestSpecies = harvestSpecies[['statecd', 'spcd']]

# sort and print the unique species
species = harvestSpecies.drop_duplicates()
species = species.sort_values('spcd')
#print(species['spcd'].unique())

# make a list of marketable species
marketSpecies = [68, 110, 111, 121, 129, 131, 132, 221, 314, 318,
                 409, 402, 403, 404, 407, 541, 544, 546, 601, 602,
                 611, 621, 651, 652, 653, 762, 802, 804, 812, 822,
                 823, 830, 832, 837, 405]

# filter the harvestSpecies to only include marketable species
harvestSpecies = harvestSpecies[harvestSpecies['spcd'].isin(marketSpecies)]
                                
# print the unique species
species = harvestSpecies.drop_duplicates()
species = species.sort_values('spcd')
#print(species['spcd'].unique())        

# how many unique species are in each state?
#print(harvestSpecies.groupby('statecd')['spcd'].nunique())

## Southern Timber Stock

In [47]:
# load biomassSouth data from the FS FIA's National Forest Inventory
# data is in excel format on the first sheet
biomassSouth = pd.read_excel('../data/Merch Bio South by spp 08-28-2024.xlsx', sheet_name=0)

# replace NaN with 0 in the biomassSouth data
biomassSouth.fillna(0, inplace=True)

# columns 13-32 are the total volume of timber in cubic feet for each size class
# for example, column 11 is '`0003 5.0-6.9'; which is size class code 0003 and size class 5.0-6.9 inches
# we can use the pandas melt function to convert these columns into rows
biomassSouth = biomassSouth.melt(
    id_vars=biomassSouth.columns[0:13],
    value_vars=biomassSouth.columns[13:29],
    var_name='size_class',
    value_name='volume'
    )

# split the size class code and size class range into two columns
biomassSouth[['size_class_code', 'size_class_range']] = \
    biomassSouth['size_class'].str.split(' ', n=1, expand=True)
# drop the first two characters of the size class code
biomassSouth['size_class_code'] = biomassSouth['size_class_code'].str[2:]
# drop the last character in the size class range
biomassSouth['size_class_range'] = biomassSouth['size_class_range'].str[:-1]

# recode evalid to add year
# convert to string of length 6 with leading zeros
biomassSouth['EVALID'] = biomassSouth['EVALID'].astype(str).str.zfill(6)

# middle two characters are the two digit year; extract and convert to 4 digit integer
biomassSouth['year'] = biomassSouth['EVALID'].str[2:4].astype(int) + 2000

# # format fips codes
# # STATECD should be two characters
# # COUNTYCD should be three characters
biomassSouth['STATECD'] = biomassSouth['STATECD'].astype(str).str.zfill(2)
biomassSouth['COUNTYCD'] = biomassSouth['COUNTYCD'].astype(str).str.zfill(3)
biomassSouth['fips'] = biomassSouth['STATECD'] + biomassSouth['COUNTYCD']

# # format survey unit codes; should be two characters
biomassSouth['UNITCD'] = biomassSouth['UNITCD'].astype(str).str.zfill(2)

# make all variables lowercase
biomassSouth.columns = biomassSouth.columns.str.lower()

# # keep only the columns we need
# # year, fips, unitcd, spclass, spcd, spgrpcd, size_class_code, size_class_range, volume
biomassSouth = biomassSouth[['year', 'statecd', 'fips', 'unitcd', 'spcd',
                                 'spgrpcd', 'spclass', 'size_class_code',
                                 'size_class_range', 'volume']]

# print number of unique species in each state
# print(biomassSouth.groupby('statecd')['spcd'].nunique())

# filter biomassSouth using harvestSpecies
biomassSouth = biomassSouth[biomassSouth['spcd'].isin(marketSpecies)]

# print number of unique species in each state
# sort and print the unique species
species = biomassSouth.groupby('statecd')['spcd'].nunique()
# print(species)


## Southern Species Dictionary

In [48]:
# Create a crosswalk dataframe
crosswalk = pd.DataFrame({'spcd': [318, 402, 403, 404, 405, 407, 409,
                                   541, 544, 546, 601, 602, 611, 621, 651,
                                   652, 653, 762, 802, 804, 812, 822, 823,
                                   830, 832, 837, 68, 110, 111, 121, 129, 131,
                                   132, 221],
                        'spname': ['sugar maple', 'bitternut hickory',
                                   'pignut hickory', 'pecan', 'shelbark hickory',
                                   'shagbark hickory', 'mockernut hickory',
                                   'white ash', 'green ash', 'blue ash',
                                   'butternut', 'black walnut', 'sweetgum',
                                   'yellow-poplar', 'cucumbertree', 'southern magnolia',
                                   'sweetbay', 'black cherry', 'white oak',
                                   'swamp white oak', 'southern red oak', 'overcup oak',
                                   'bur oak', 'pin oak', 'chestnut oak', 'black oak',
                                   'eastern redcedar', 'shortleaf pine', 'slash pine',
                                   'longleaf pine', 'eastern white pine', 'loblolly pine',
                                   'Virginia pine', 'baldcypress']
                          })


## Price Regions

In [49]:
# create a crosswalk to merge the biomassSouth data with the pricesSouth data
crosswalkSpecies = {'Softwood': 'Pine',
                    'Hardwood': 'Oak'}

# map priceSpecies to spclass
biomassSouth['priceSpecies'] = biomassSouth['spclass'].map(crosswalkSpecies)
# product crosswalk
crosswalkProduct = {'0003': 'Pulpwood',
                    '0004': 'Pulpwood',
                    '0005': 'Pulpwood',
                    '0006': 'Pulpwood',
                    '0007': 'Sawtimber',
                    '0008': 'Sawtimber',
                    '0009': 'Sawtimber',
                    '0010': 'Sawtimber',
                    '0011': 'Sawtimber',
                    '0012': 'Sawtimber',
                    '0013': 'Sawtimber',
                    '0014': 'Sawtimber',
                    '0015': 'Sawtimber',
                    '0016': 'Sawtimber',
                    '0017': 'Sawtimber',
                    '0018': 'Sawtimber'}

# map spclass to product
biomassSouth['Product'] = biomassSouth['size_class_code'].map(crosswalkProduct)
# priceRegions
# print column names
# print(pricesSouth.columns)

# read in spatial crosswalk table
priceRegions = pd.read_csv('../data/priceRegions.csv')
# convert columns to character
priceRegions['fips'] = priceRegions['fips'].astype(str).str.zfill(5)
priceRegions['unitcd'] = priceRegions['unitcd'].fillna(0).astype(int)
priceRegions['unitcd'] = priceRegions['unitcd'].astype(str).str.zfill(2)
priceRegions['statecd'] = priceRegions['statecd'].astype(str).str.zfill(2)
priceRegions['priceRegion'] = priceRegions['priceRegion'].astype(str).str.zfill(2)

# drop fips from priceRegions by keeping only unique values of statecd and priceRegion and unitcd
priceRegions = priceRegions.drop(columns=['fips']).drop_duplicates()
# print(priceRegions.head())

## Southern Table

In [50]:
# drop states in biomassSouth that are not in pricesSouth
biomassSouth = biomassSouth[biomassSouth['statecd'].isin(pricesSouth['statecd'].unique())]

# print column names
# print(biomassSouth.columns)


# reduce biomassSouth on fips, unitcd, priceRegion,
#  Product, priceSpecies, size_class_range
biomassSouth = biomassSouth.groupby(['statecd', 'fips', 'unitcd', 'spclass',
                                    'spcd', 'spgrpcd', 'Product', 'priceSpecies',
                                    'size_class_range', 'size_class_code'])['volume'].mean().reset_index()

biomassSouth = biomassSouth.groupby(['statecd', 'unitcd', 'Product', 'priceSpecies',
                                     'spcd', 'spgrpcd', 'size_class_range',
                                    'size_class_code'])['volume'].sum().reset_index()



# merge pricesSouth with priceRegions
tableSouth = pd.merge(pricesSouth, priceRegions,
                        how='left', on=['statecd', 'priceRegion'])
# merge biomassSouth with tableSouth
tableSouth = pd.merge(biomassSouth, tableSouth,
                        how='left', on=['statecd', 'unitcd', 'Product', 'priceSpecies'])

# calculate total value as volume * price
tableSouth['value'] = tableSouth['volume'] * tableSouth['cuftPrice']


# in tableSouth, replace spcd with spname
tableSouth = pd.merge(tableSouth, crosswalk, on='spcd', how='left')
# rename spname to species
tableSouth.rename(columns={'priceSpecies': 'spclass'}, inplace=True)

# in the column, spclass, replace 'Pine' with 'Softwood' and 'Oak' with 'Hardwood'
tableSouth['spclass'] = tableSouth['spclass'].replace({'Pine': 'Coniferous',
                                                        'Oak': 'Non-coniferous'})

columnOrder = ['stateAbbr', 'statecd', 'unitcd', 'priceRegion', 'spcd', 'spname', 'spgrpcd',
                'spclass', 'Product', 'size_class_code', 'size_class_range',
                'cuftPrice', 'volume', 'value']

tableSouth = tableSouth[columnOrder]

# sort the table by statecd, unitcd, priceRegion, spcd, spgrpcd, priceSpecies, Product, then size_class_code
tableSouth = tableSouth.sort_values(['statecd', 'unitcd', 'priceRegion',
                                    'spcd', 'spgrpcd', 'spclass', 'Product', 'size_class_code'])



## Physical and Monetary Tables

In [51]:

tablePhysical = tableSouth.copy()
# drop cuftPrice and value columns
tablePhysical = tablePhysical.drop(columns=['cuftPrice', 'value'])

tableMonetary = tableSouth.copy()
# drop volume column
tableMonetary = tableMonetary.drop(columns=['volume'])


# write tablePhysical to a csv file
tablePhysical.to_csv('../data/tablePhysicalSouth.csv', index=False)
# write tableMonetary to a csv file
tableMonetary.to_csv('../data/tableMonetarySouth.csv', index=False)


## Aggregate to Pulpwood/Sawtimber

In [7]:

# if size_class_code is in ['0003', '0004', '0005', '0006'], save to pwTable
pwTable = tableSouth[tableSouth['size_class_code'].isin(['0003', '0004', '0005', '0006'])]

# slice pwTable for Product = 'Pulpwood'
pwTable = pwTable[pwTable['Product'] == 'Pulpwood']

# drop columns from pwTable
pwTable = pwTable[['statecd', 'spcd', 'priceSpecies', 'volume', 'value']]

# rename columns
pwTable.rename(columns={'spcd': 'species',
                        'priceSpecies': 'spclass',
                        'volume': 'pwVolume',
                        'value': 'pwValue'}, inplace=True)

# reduce pwTable to the sum of pwVolume and pwValue by statecd, spclass, species
pwTable = pwTable.groupby(['statecd', 'spclass', 'species']).agg({
    'pwVolume': 'sum',
    'pwValue': 'sum'
}).reset_index()


# if size_class_code is in ['0007',...], save to stTable
stTable = tableSouth[tableSouth['size_class_code'].isin(['0007', '0008', '0009',
                                                         '0010', '0011', '0012',
                                                         '0013', '0014', '0015',
                                                         '0016', '0017', '0018'])]

# drop columns from stTable
stTable = stTable[['statecd', 'spcd', 'priceSpecies', 'volume', 'value']]

# rename columns
stTable.rename(columns={'spcd': 'species',
                        'priceSpecies': 'spclass',
                        'volume': 'stVolume',
                        'value': 'stValue'}, inplace=True)


# reduce stTable to the sum of stVolume and stValue by statecd, spclass, species
stTable = stTable.groupby(['statecd', 'spclass', 'species']).agg({
    'stVolume': 'sum',
    'stValue': 'sum'
}).reset_index()

table_by_product = pd.merge(pwTable, stTable, on=['statecd', 'spclass', 'species'])



In [26]:
# group by priceSpecies and product and use agg() to get summary statistics
# round to two decimal places
# sort by size_class_range
summary = table.groupby(['spclass', 'species']).agg({
    'pwVolume': 'sum',
    'pwValue': 'sum',
    'stVolume': 'sum',
    'stValue': 'sum'
}).reset_index()


# Convert volume and value to billions
summary['pwVolume'] = summary['pwVolume'] / 1000000000
summary['pwValue'] = summary['pwValue'] / 1000000000
summary['stVolume'] = summary['stVolume'] / 1000000000
summary['stValue'] = summary['stValue'] / 1000000000

# Round to two decimal places
summary = summary.round(2)

# Sort by priceSpecies, Product, and size_class_code
summary = summary.sort_values(by=['spclass', 'species'])

# add a column for total volume and total value
summary['totalVolume'] = summary['pwVolume'] + summary['stVolume']
summary['totalValue'] = summary['pwValue'] + summary['stValue']

# Sort by total value
summary = summary.sort_values('totalValue', ascending=False)

# save to csv
summary.to_csv('../data/final-table-v01.csv', index=False)
tableGL.to_csv('../data/tableGL.csv', index=False)
tableSouth.to_csv('../data/tableSouth.csv', index=False)
table.to_csv('../data/table.csv', index=False)

# Display the summary statistics
summary

,spclass,species,pwVolume,pwValue,stVolume,stValue,totalVolume,totalValue
16,Hardwood,hard maple,1039.01,0.00,852.65,36200.57,1891.66,36200.57
25,Hardwood,red oak,277.17,0.00,807.65,22025.61,1084.82,22025.61
28,Hardwood,soft maple,935.51,0.00,597.29,9258.19,1532.80,9258.19
45,Softwood,loblolly pine,1280.84,739.93,1100.50,6372.24,2381.34,7112.17
37,Hardwood,white oak,255.86,76.57,656.67,6669.26,912.53,6745.83
20,Hardwood,oak,266.77,22.42,572.97,4932.52,839.74,4954.94
1,Hardwood,aspen,1242.08,291.44,600.87,4143.31,1842.95,4434.75
48,Softwood,red pine,400.57,173.02,322.85,3937.31,723.42,4110.33
39,Hardwood,yellow-poplar,181.99,87.10,495.21,3943.57,677.20,4030.67
52,Softwood,white pine,152.23,0.00,369.71,3462.74,521.94,3462.74


In [196]:
# what is the total volume of the timber stock in the southern region?
print(f"The total volume of the timber stock in the southern region is {summarySouth['volume'].sum():,.2f} billion cubic feet")

# what is the total value of the timber stock in the southern region?
print(f"The total value of the timber stock in the southern region is ${summarySouth['value'].sum():,.2f} (in billions)")

# what is the total volume of the timber stock in the northern region?
print(f"The total volume of the timber stock in the great lakes region is {summaryGL['pwVolume'].sum() + summaryGL['stVolume'].sum():,.2f} billion cubic feet")

# what is the total value of the timber stock in the northern region?
print(f"The total value of the timber stock in the great lakes region is ${summaryGL['pwValue'].sum() + summaryGL['stValue'].sum():,.2f} (in billions)")

The total volume of the timber stock in the southern region is 6,230.35 billion cubic feet
The total value of the timber stock in the southern region is $25,945.71 (in billions)
The total volume of the timber stock in the great lakes region is 11,466.59 billion cubic feet
The total value of the timber stock in the great lakes region is $98,850.94 (in billions)
